# Homework 6: Web Applications

For this homework, you're going to write a web API for the lake data in the MONDIAL database. (Make sure you've imported the data as originally outlined in [our week 1 tutorial](https://github.com/ledeprogram/data-and-databases/blob/master/SQL_notes.md).)

The API should perform the following tasks:

* A request to `/lakes` should return a JSON list of dictionaries, with the information from the `name`, `elevation`, `area` and `type` fields from the `lake` table in MONDIAL.
* The API should recognize the query string parameter `sort`. When left blank or set to `name`, the results should be sorted by the name of the lake (in alphabetical order). When set to `area` or `elevation`, the results should be sorted by the requested field, in *descending* order.
* The API should recognize the query string parameter `type`. When specified, the results should *only* include rows that have the specified value in the `type` field.
* You should be able to use *both* the `sort` and `type` parameters in any request.

This notebook contains only *test requests to your API*. Write the API as a standalone Python program, start the program and then run the code in the cells below to ensure that your API produces the expected output. When you're done, paste the source code in the final cell (so we can check your work, if needed).

Hints when writing your API code:

* You'll need to construct the SQL query as a string, piece by piece. This will likely involve a somewhat messy tangle of `if` statements. Lean into the messy tangle.
* Make sure to use parameter placeholders (%s) in the query.
* If you're getting SQL errors, print out your SQL statement in the request handler function so you can debug it. (When you use `print()` in Flask, the results will display in your terminal window.)
* When in doubt, return to the test code. Examine it carefully and make sure you know exactly what it's trying to do.

## Problem set #1: A list of lakes

Your API should return a JSON list of dictionaries (objects). Use the code below to determine what the keys of the dictionaries should be. (For brevity, this example only prints out the first ten records, but of course your API should return all of them.)

Expected output:

    143 lakes
    Ammersee - elevation: 533 m / area: 46 km^2 / type: None
    Arresoe - elevation: None m / area: 40 km^2 / type: None
    Atlin Lake - elevation: 668 m / area: 798 km^2 / type: None
    Balaton - elevation: 104 m / area: 594 km^2 / type: None
    Barrage de Mbakaou - elevation: None m / area: None km^2 / type: dam
    Bodensee - elevation: 395 m / area: 538 km^2 / type: None
    Brienzersee - elevation: 564 m / area: 29 km^2 / type: None
    Caspian Sea - elevation: -28 m / area: 386400 km^2 / type: salt
    Chad Lake - elevation: 250 m / area: 23000 km^2 / type: salt
    Chew Bahir - elevation: 520 m / area: 800 km^2 / type: salt

In [1]:
import requests
data = requests.get('http://localhost:5000/lakes').json()
print(len(data), "lakes")
for item in data[:10]:
    print(item['name'], "- elevation:", item['elevation'], "m / area:", item['area'], "km^2 / type:", item['type'])

143 lakes
Ammersee - elevation: 533.0 m / area: 46.6 km^2 / type: None
Arresoe - elevation: None m / area: 40.2 km^2 / type: None
Atlin Lake - elevation: 668.0 m / area: 798.0 km^2 / type: None
Balaton - elevation: 104.0 m / area: 594.0 km^2 / type: None
Barrage de Mbakaou - elevation: None m / area: None km^2 / type: dam
Bodensee - elevation: 395.0 m / area: 538.5 km^2 / type: None
Brienzersee - elevation: 564.0 m / area: 29.8 km^2 / type: None
Caspian Sea - elevation: -28.0 m / area: 386400.0 km^2 / type: salt
Chad Lake - elevation: 250.0 m / area: 23000.0 km^2 / type: salt
Chew Bahir - elevation: 520.0 m / area: 800.0 km^2 / type: salt


## Problem set #2: Lakes of a certain type

The following code fetches all lakes of type `salt` and finds their average area and elevation.

Expected output:

    average area: 18880
    average elevation: 970

In [2]:
import requests
data = requests.get('http://localhost:5000/lakes?type=salt').json()
avg_area = sum([x['area'] for x in data if x['area'] is not None]) / len(data)
avg_elev = sum([x['elevation'] for x in data if x['elevation'] is not None]) / len(data)
print("average area:", int(avg_area))
print("average elevation:", int(avg_elev))

average area: 18880
average elevation: 970


## Problem set #3: Lakes in order

The following code fetches lakes in reverse order by their elevation and prints out the name of the first fifteen, excluding lakes with an empty elevation field.

Expected output:

    * Licancabur Crater Lake
    * Nam Co
    * Lago Junin
    * Lake Titicaca
    * Poopo
    * Salar de Uyuni
    * Koli Sarez
    * Lake Irazu
    * Qinghai Lake
    * Segara Anak
    * Lake Tahoe
    * Crater Lake
    * Lake Tana
    * Lake Van
    * Issyk-Kul

In [3]:
import requests
data = requests.get('http://localhost:5000/lakes?sort=elevation').json()
for item in [x['name'] for x in data if x['elevation'] is not None][:15]:
    print("*", item)

* Licancabur Crater Lake
* Nam Co
* Lago Junin
* Lake Titicaca
* Poopo
* Salar de Uyuni
* Koli Sarez
* Lake Irazu
* Qinghai Lake
* Segara Anak
* Lake Tahoe
* Crater Lake
* Lake Tana
* Lake Van
* Issyk-Kul


## Problem set #4: Order and type

The following code prints the names of the largest caldera lakes, ordered in reverse order by area.

Expected output:

    * Lake Nyos
    * Lake Toba
    * Lago Trasimeno
    * Lago di Bolsena
    * Lago di Bracciano
    * Crater Lake
    * Segara Anak
    * Laacher Maar

In [4]:
import requests
data = requests.get('http://localhost:5000/lakes?sort=area&type=caldera').json()
for item in data:
    print("*", item['name'])

* Lake Nyos
* Lake Toba
* Lago Trasimeno
* Lago di Bolsena
* Lago di Bracciano
* Crater Lake
* Segara Anak
* Laacher Maar


## Problem set #5: Error handling

Your API should work fine even when faced with potential error-causing inputs. For example, the expected output for this statement is an empty list (`[]`), *not* every row in the table.

In [5]:
import requests
data = requests.get('http://localhost:5000/lakes', params={'type': "' OR true; --"}).json()
data

[{'area': 1040.0, 'elevation': 119.0, 'name': 'Inari', 'type': 'None'},
 {'area': 928.0, 'elevation': 123.0, 'name': 'Oulujaervi', 'type': 'None'},
 {'area': 472.0, 'elevation': None, 'name': 'Kallavesi', 'type': 'None'},
 {'area': 4370.0, 'elevation': 76.0, 'name': 'Saimaa', 'type': 'None'},
 {'area': 1118.0, 'elevation': 78.0, 'name': 'Paeijaenne', 'type': 'None'},
 {'area': 368.0, 'elevation': 123.0, 'name': 'Mjoesa-See', 'type': 'None'},
 {'area': 173.0, 'elevation': 419.0, 'name': 'Storuman', 'type': 'None'},
 {'area': 290.0, 'elevation': 161.0, 'name': 'Siljan', 'type': 'impact'},
 {'area': 1140.0, 'elevation': 0.7, 'name': 'Maelaren', 'type': 'None'},
 {'area': 5648.0, 'elevation': 44.0, 'name': 'Vaenern', 'type': 'None'},
 {'area': 1900.0, 'elevation': 88.0, 'name': 'Vaettern', 'type': 'None'},
 {'area': 40.2, 'elevation': None, 'name': 'Arresoe', 'type': 'None'},
 {'area': 56.0, 'elevation': 16.0, 'name': 'Loch Ness', 'type': 'None'},
 {'area': 71.0, 'elevation': 7.9, 'name': 

Specifying a field other than `name`, `area` or `elevation` for the `sort` parameter should fail silently, defaulting to sorting alphabetically. Expected output: `['Ammersee', 'Arresoe', 'Atlin Lake', 'Balaton', 'Barrage de Mbakaou']
`

In [6]:
import requests
data = requests.get('http://localhost:5000/lakes', params={'sort': "florb"}).json()
[x['name'] for x in data[:5]]

['Ammersee', 'Arresoe', 'Atlin Lake', 'Balaton', 'Barrage de Mbakaou']

## Paste your code

Please paste the code for your entire Flask application in the cell below, in case we want to take a look when grading or debugging your assignment.

In [ ]:
## THIS CODE RETURNS CORRECTLY ALL OF THE ABOVE EXCEPT THE FIRST PROBLEM OF PROBLEM SET 5, there are two lines commented out below that I used 
# to try solve that  but it didn't work out unfortunately.

from flask import Flask, request, jsonify
import pg8000

app = Flask (__name__)
conn = pg8000.connect(database="mondial", user="gcg")

@app.route("/lakes")
def get_lakes():
    cursor = conn.cursor()
    sort_default = "name"
    sorting_option = request.args.get('sort', sort_default)
    if 'area' in sorting_option:
        sorting_option = sorting_option + " DESC"
    if 'elevation' in sorting_option:
        sorting_option = sorting_option + " DESC"
    if 'florb' in sorting_option:
        sorting_option = sort_default

    cursor.execute("SELECT name, elevation, area, type FROM lake ORDER BY {}".format(sorting_option))


    type_option = request.args.get('type', None)
    # if "' " in type_option:
    #     return ([])
    if type_option:
        cursor.execute("SELECT name, elevation, area, type FROM lake WHERE type = '{}' ORDER BY {}".format(type_option, sorting_option))
    else:
        cursor.execute("SELECT name, elevation, area, type FROM lake ORDER BY {}".format(sorting_option))


    output=[]
    for item in cursor.fetchall():
        get_name = str(item[0])
        get_type = str(item[3])
        if item[1] is not None:
            get_elevation = float(item[1])
        else:
            get_elevation = None
        if item[2] is not None:
            get_area = float(item[2])
        else:
            get_area = None
        output.append({'name': get_name,
                       'elevation': get_elevation,
                       'area': get_area,
                       'type': get_type})

    return jsonify(output)

app.run()